In [1]:
import numpy as np
import pandas as pd
from random import sample

import hyperopt
from hyperopt import hp, Trials
from hyperopt.pyll.base import scope

from sklearn.metrics import f1_score, log_loss, accuracy_score
from sklearn.utils.class_weight import compute_sample_weight

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

from coremdlr.models import NetworkModel
from coremdlr.config import defaults

from coremdlr.datasets import WellLoader, FaciesDataset
from coremdlr.datasets.utils import infer_test_wells

import matplotlib.pyplot as plt
%matplotlib inline

/home/ross/anaconda3/envs/core-dev/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ross/anaconda3/envs/core-dev/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ross/anaconda3/envs/core-dev/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ross/anaconda3/envs/core-dev/lib/python3

In [2]:
WAVENET_SEARCH_SPACE = {
    'feature' : 'pseudoGR',
    'network' : 'wavenet',
    'summary' : False,
    'sequence_size' : 128,
    'network_args' : {
        'num_blocks' : scope.int(hp.quniform('num_blocks', 2, 20, 2)),
        'block_filters' : scope.int(hp.quniform('block_filters', 16, 256, 16)),
        'residual_filters' : scope.int(hp.quniform('residual_filters', 8, 128, 8)),
        'output_resolution' : 32,
        'dropout_rate' : 0.5,
    },
    'optimizer_args' : {
        'optimizer' : 'Adam',
    },
    'loss' : {'categorical_crossentropy': hp.uniform('cat_loss', 0.1, 1.0),
              'ordinal_squared_error': hp.uniform('ord_loss', 0.1, 1.0)}
}


fit_args = {
    'batch_size' : 16,
    'epochs' : 20,
    'class_weighted' : True
}
CHKPT_FNAME = 'temp_best_weights.h5'


def make_params_usable(params):
    """
    Convert params returned by hyperopt to usable `model_args`
    """
    p = {**WAVENET_SEARCH_SPACE, **params}
    p['network_args']['num_blocks'] = int(params['network_args']['num_blocks'])
    p['network_args']['block_filters'] = int(params['network_args']['block_filters'])
    p['network_args']['residual_filters'] = int(params['network_args']['residual_filters'])
    return p

In [3]:
def run_experiment(num_train, max_evals=15, search_space=WAVENET_SEARCH_SPACE):
    
    assert num_train in [3, 6, 9], 'Must use one of predefined train/test splits'
    
    train_sets = getattr(defaults, f'TRAIN_SETS_{num_train}')
    
    datasets = [FaciesDataset(s, infer_test_wells(s), features=["pseudoGR"],
                              pseudoGR_args={'features' : ['mean', 'var'], 
                                             'per_channel' : True}) for s in train_sets[:2]]
    
    for dset in datasets:
        dset.load_or_generate_data()
    
    def train_model(model_config):
        
        acc_scores, f1_scores, log_losses = [], [], []
        
        for dset in datasets:
            
            model = NetworkModel(dset, model_args=model_config)
            
            chkpt_callback = ModelCheckpoint(CHKPT_FNAME, monitor='val_loss', verbose=1,
                                        save_best_only=True, 
                                        save_weights_only=True)
            earlystop_callback = EarlyStopping(monitor='val_loss', patience=3)
            fit_args['callbacks'] = [chkpt_callback, earlystop_callback]
            
            print(f'Training with config: {model_config}')
            
            model.fit(dset, **fit_args)
            
            model.network.load_weights(CHKPT_FNAME)
            
            y_proba = model.predict_proba(dset.X_test)
            y_pred = np.argmax(y_proba, -1)
            
            acc_scores.append(accuracy_score(dset.y_test, y_pred))
            
            f1_scores.append(f1_score(dset.y_test, y_pred, average='macro'))
            
            log_losses.append(log_loss(dset.y_test, y_proba, 
                                      sample_weight=compute_sample_weight('balanced', dset.y_test)))
            
        return {'loss' : np.mean(log_losses),
                'acc_scores' : acc_scores,
                'f1_scores' : f1_scores,
                'log_losses' : log_losses,
                'status' : hyperopt.STATUS_OK}
    
    
    trials = Trials()
    
    best_params = hyperopt.fmin(
        fn=train_model,
        space=WAVENET_SEARCH_SPACE,
        algo=hyperopt.rand.suggest,
        max_evals=max_evals,
        trials=trials
    )
    
    return trials, best_params

In [ ]:
trails3, best_params3 = run_experiment(3)

Loading Well:  204-24a-7  from  /home/ross/Dropbox/core_data/facies/train_data


/home/ross/code/python/coremdlr/coremdlr/datasets/utils.py:88: RuntimeWarning: Mean of empty slice
  output_features.append(np.nanmean(img, axis=1))
/home/ross/code/python/coremdlr/coremdlr/datasets/utils.py:92: RuntimeWarning: Degrees of freedom <= 0 for slice.
  output_features.append(np.nanvar(img, axis=1))


Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (19294,)), ('top', (19294,)), ('base', (19294,)), ('pseudoGR', (19294, 32, 8))]
Loading Well:  204-24a-6  from  /home/ross/Dropbox/core_data/facies/train_data
Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (13034,)), ('top', (13034,)), ('base', (13034,)), ('pseudoGR', (13034, 32, 8))]
Loading Well:  204-20-3  from  /home/ross/Dropbox/core_data/facies/train_data
Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (10906,)), ('top', (10906,)), ('base', (10906,)), ('pseudoGR', (10906, 32, 8))]
Loading Well:  204-20-2  from  /home/ross/Dropbox/core_data/facies/train_data


/home/ross/code/python/coremdlr/coremdlr/datasets/utils.py:88: RuntimeWarning: Mean of empty slice
  output_features.append(np.nanmean(img, axis=1))
/home/ross/code/python/coremdlr/coremdlr/datasets/utils.py:92: RuntimeWarning: Degrees of freedom <= 0 for slice.
  output_features.append(np.nanvar(img, axis=1))


Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (2940,)), ('top', (2940,)), ('base', (2940,)), ('pseudoGR', (2940, 32, 8))]
Loading Well:  204-20-6a  from  /home/ross/Dropbox/core_data/facies/train_data
Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (3542,)), ('top', (3542,)), ('base', (3542,)), ('pseudoGR', (3542, 32, 8))]
Loading Well:  204-19-6  from  /home/ross/Dropbox/core_data/facies/train_data
Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (1981,)), ('top', (1981,)), ('base', (1981,)), ('pseudoGR', (1981, 32, 8))]
Loading Well:  204-19-7  from  /home/ross/Dropbox/core_data/facies/train_data
Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (9807,)), ('top', (9807,)), ('base', (9807,)), ('

In [9]:
trails3.results

[{'loss': 1.2823822840234045,
  'acc_scores': [0.6471350628174843, 0.5788006607279806],
  'f1_scores': [0.4380973875686694, 0.4277482009530229],
  'log_losses': [1.2016444932465329, 1.3631200748002759],
  'status': 'ok'},
 {'loss': 1.299784543059565,
  'acc_scores': [0.6316449610134415, 0.5061648280337443],
  'f1_scores': [0.5237932959522477, 0.2977448632334879],
  'log_losses': [1.136800029226427, 1.4627690568927028],
  'status': 'ok'},
 {'loss': 1.3270778951968145,
  'acc_scores': [0.6475949578777933, 0.5397026724087074],
  'f1_scores': [0.4448563606195015, 0.39340167170010154],
  'log_losses': [1.211353905495945, 1.4428018848976842],
  'status': 'ok'},
 {'loss': 1.3282230119479461,
  'acc_scores': [0.6471768714593307, 0.5026842074213911],
  'f1_scores': [0.4399909311417751, 0.3186784968869391],
  'log_losses': [1.2588568103176998, 1.3975892135781922],
  'status': 'ok'},
 {'loss': 1.4324349369692504,
  'acc_scores': [0.6533018374898092, 0.3323845200873105],
  'f1_scores': [0.44145895

In [10]:
best_params3

{'block_filters': 208.0,
 'cat_loss': 0.838840319697992,
 'num_blocks': 8.0,
 'ord_loss': 0.11126222976973828,
 'residual_filters': 40.0}

In [11]:
trails3.best_trial

{'state': 2,
 'tid': 10,
 'spec': None,
 'result': {'loss': 1.2290156809122172,
  'acc_scores': [0.6283211739866631, 0.5711609934517138],
  'f1_scores': [0.5014605527975033, 0.42513150268780686],
  'log_losses': [1.1241791624602282, 1.3338521993642063],
  'status': 'ok'},
 'misc': {'tid': 10,
  'cmd': ('domain_attachment', 'FMinIter_Domain'),
  'workdir': None,
  'idxs': {'block_filters': [10],
   'cat_loss': [10],
   'num_blocks': [10],
   'ord_loss': [10],
   'residual_filters': [10]},
  'vals': {'block_filters': [208.0],
   'cat_loss': [0.838840319697992],
   'num_blocks': [8.0],
   'ord_loss': [0.11126222976973828],
   'residual_filters': [40.0]}},
 'exp_key': None,
 'owner': None,
 'version': 0,
 'book_time': datetime.datetime(2019, 7, 10, 15, 53, 58, 839000),
 'refresh_time': datetime.datetime(2019, 7, 10, 16, 1, 6, 627000)}

In [3]:
def pred_results_df(num_train, model_config):
    
    assert num_train in [3, 6, 9], 'Must use one of predefined train/test splits'
    
    train_sets = getattr(defaults, f'TRAIN_SETS_{num_train}')
    
    pred_dfs = []
    
    for i, train_set in enumerate(train_sets):
        dset = FaciesDataset(train_set, infer_test_wells(train_set), features=['pseudoGR'], 
                             pseudoGR_args={'features' : ['mean', 'var'], 'per_channel' : True})
        
        dset.load_or_generate_data()
        
        model = NetworkModel(dset, model_args=model_config)
        
        chkpt_callback = ModelCheckpoint(CHKPT_FNAME, monitor='val_loss', verbose=1,
                                        save_best_only=True, 
                                        save_weights_only=True)
        
        model.fit(dset, callbacks=[chkpt_callback], verbose=False, **fit_args)
        
        model.network.load_weights(CHKPT_FNAME)
        
        #print('predict_proba: ', model.predict_proba(dset.X_test))
        
        for test_well_name in dset.test_well_names:
            df = model.preds_dataframe(test_well_name)
            df['well'] = test_well_name
            df['set_number'] = i
            pred_dfs.append(df)
        
    return pd.concat(pred_dfs)

In [4]:
params = {
    'feature' : 'pseudoGR',
    'network' : 'wavenet',
    'summary' : False,
    'sequence_size' : 128,
    'network_args' : {
        'num_blocks' : 8,
        'block_filters' : 208,
        'residual_filters' : 40,
        'output_resolution' : 32,
        'dropout_rate' : 0.5,
    },
    'optimizer_args' : {
        'optimizer' : 'Adam'
    }
}

preds_df_3 = pred_results_df(3, params)
preds_df_3.to_csv('pgr_wavenet_results_train_sets3.csv')

Loading Well:  204-24a-7  from  /home/ross/Dropbox/core_data/facies/train_data


/home/ross/code/python/coremdlr/coremdlr/datasets/utils.py:88: RuntimeWarning: Mean of empty slice
  output_features.append(np.nanmean(img, axis=1))
/home/ross/code/python/coremdlr/coremdlr/datasets/utils.py:92: RuntimeWarning: Degrees of freedom <= 0 for slice.
  output_features.append(np.nanvar(img, axis=1))


Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (19294,)), ('top', (19294,)), ('base', (19294,)), ('pseudoGR', (19294, 32, 8))]
Loading Well:  204-24a-6  from  /home/ross/Dropbox/core_data/facies/train_data
Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (13006,)), ('top', (13006,)), ('base', (13006,)), ('pseudoGR', (13006, 32, 8))]
Loading Well:  204-20-3  from  /home/ross/Dropbox/core_data/facies/train_data
Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (10906,)), ('top', (10906,)), ('base', (10906,)), ('pseudoGR', (10906, 32, 8))]
Loading Well:  204-19-3A  from  /home/ross/Dropbox/core_data/facies/train_data


/home/ross/code/python/coremdlr/coremdlr/datasets/utils.py:88: RuntimeWarning: Mean of empty slice
  output_features.append(np.nanmean(img, axis=1))
/home/ross/code/python/coremdlr/coremdlr/datasets/utils.py:92: RuntimeWarning: Degrees of freedom <= 0 for slice.
  output_features.append(np.nanvar(img, axis=1))


Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (9590,)), ('top', (9590,)), ('base', (9590,)), ('pseudoGR', (9590, 32, 8))]
Loading Well:  205-21b-3  from  /home/ross/Dropbox/core_data/facies/train_data
Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (3842,)), ('top', (3842,)), ('base', (3842,)), ('pseudoGR', (3842, 32, 8))]
Loading Well:  204-20-1Z  from  /home/ross/Dropbox/core_data/facies/train_data
Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (1917,)), ('top', (1917,)), ('base', (1917,)), ('pseudoGR', (1917, 32, 8))]
Loading Well:  204-20a-7  from  /home/ross/Dropbox/core_data/facies/train_data
Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (3105,)), ('top', (3105,)), ('base', (3105,)), 

...

Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (1917,)), ('top', (1917,)), ('base', (1917,)), ('pseudoGR', (1917, 32, 8))]
Loading Well:  204-20a-7  from  /home/ross/Dropbox/core_data/facies/train_data
Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (3105,)), ('top', (3105,)), ('base', (3105,)), ('pseudoGR', (3105, 32, 8))]
Loading Well:  204-24a-6  from  /home/ross/Dropbox/core_data/facies/train_data
Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (13006,)), ('top', (13006,)), ('base', (13006,)), ('pseudoGR', (13006, 32, 8))]
Loading Well:  204-19-6  from  /home/ross/Dropbox/core_data/facies/train_data
Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (1884,)), ('top', (1884,)), ('base', (1884,)

/home/ross/code/python/coremdlr/coremdlr/datasets/utils.py:88: RuntimeWarning: Mean of empty slice
  output_features.append(np.nanmean(img, axis=1))
/home/ross/code/python/coremdlr/coremdlr/datasets/utils.py:92: RuntimeWarning: Degrees of freedom <= 0 for slice.
  output_features.append(np.nanvar(img, axis=1))


Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (11210,)), ('top', (11210,)), ('base', (11210,)), ('pseudoGR', (11210, 32, 8))]
Loading Well:  204-20-2  from  /home/ross/Dropbox/core_data/facies/train_data
Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (2940,)), ('top', (2940,)), ('base', (2940,)), ('pseudoGR', (2940, 32, 8))]
Loading Well:  204-19-6  from  /home/ross/Dropbox/core_data/facies/train_data
Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (1884,)), ('top', (1884,)), ('base', (1884,)), ('pseudoGR', (1884, 32, 8))]
Loading Well:  204-19-3A  from  /home/ross/Dropbox/core_data/facies/train_data


/home/ross/code/python/coremdlr/coremdlr/datasets/utils.py:88: RuntimeWarning: Mean of empty slice
  output_features.append(np.nanmean(img, axis=1))
/home/ross/code/python/coremdlr/coremdlr/datasets/utils.py:92: RuntimeWarning: Degrees of freedom <= 0 for slice.
  output_features.append(np.nanvar(img, axis=1))


Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (9590,)), ('top', (9590,)), ('base', (9590,)), ('pseudoGR', (9590, 32, 8))]
Loading Well:  205-21b-3  from  /home/ross/Dropbox/core_data/facies/train_data
Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (3842,)), ('top', (3842,)), ('base', (3842,)), ('pseudoGR', (3842, 32, 8))]
Loading Well:  204-20-1Z  from  /home/ross/Dropbox/core_data/facies/train_data
Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (1917,)), ('top', (1917,)), ('base', (1917,)), ('pseudoGR', (1917, 32, 8))]
Loading Well:  204-20a-7  from  /home/ross/Dropbox/core_data/facies/train_data
Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (3105,)), ('top', (3105,)), ('base', (3105,)), 

In [5]:
accuracy_score(preds_df_3.y_pred.values, preds_df_3.y_true.values)

0.5544152276295133